# SoyNLP

In [3]:
from soynlp import DoublespaceLineCorpus

#문서 단위 말뭉치 생성
corups = DoublespaceLineCorpus("2016-10-20.txt", iter_sent = True)
len(corups)

223357

In [4]:
from soynlp.word import WordExtractor

word_extractor = WordExtractor()

word_extractor = WordExtractor()
word_extractor.train(corups)

training was done. used memory 0.658 Gbse memory 0.704 Gb


In [5]:
word_score = word_extractor.extract()


all cohesion probabilities was computed. # words = 223348
all branching entropies was computed # words = 360721
all accessor variety was computed # words = 360721


In [6]:
# word_score["연합"].cohesion_forward
word_score["연합뉴"].cohesion_forward
# word_score["연합뉴스"].cohesion_forward

np.float64(0.43154839105434084)

In [21]:
from soynlp.tokenizer import LTokenizer
scores = {word:score.cohesion_forward for word, score in word_score.items()}
l_tokenizer = LTokenizer(scores = scores)
l_tokenizer.tokenize("연합뉴스는 정말로 좋은 언론사이다")

['연합뉴스', '는', '정말로', '좋은', '언론', '사이다']

# Call API

In [1]:
import time
from googleapiclient.discovery import build
import isodate
import json
import os

# 환경 변수에서 API 키 가져오기
API_KEY = os.getenv("YOUTUBE_API_KEY")
if not API_KEY:
    raise ValueError("API 키가 설정되지 않았습니다. 환경 변수 'YOUTUBE_API_KEY'를 설정하세요.")

# YouTube API 설정
youtube = build("youtube", "v3", developerKey=API_KEY)

# 카테고리 ID 설정
CATEGORIES = {
    "News & Politics": "25"
}

# 동영상 데이터 가져오기
def fetch_trending_videos(category_id, region_code="KR", max_results=200):
    videos = []
    next_page_token = None

    while len(videos) < max_results:
        try:
            request = youtube.videos().list(
                part="snippet,statistics,contentDetails",
                chart="mostPopular",
                regionCode=region_code,
                videoCategoryId=category_id,
                maxResults=min(50, max_results - len(videos)),
                pageToken=next_page_token
            )
            response = request.execute()

            for item in response.get("items", []):
                duration = isodate.parse_duration(item["contentDetails"]["duration"])
                duration_in_seconds = duration.total_seconds()  #초로 바꾸기기

                if duration_in_seconds > 80:  # 80초 이상의 동영상만 가져오기
                    videos.append({
                        "video_id": item["id"],
                        "title": item["snippet"]["title"],
                        "description": item["snippet"]["description"],
                        "tags": item["snippet"].get("tags", []),
                        "duration": str(duration),
                        "view_count": int(item["statistics"].get("viewCount", 0)),
                        "like_count": int(item["statistics"].get("likeCount", 0)),
                        "comment_count": int(item["statistics"].get("commentCount", 0)),
                        "category_id": category_id,
                    })

            next_page_token = response.get("nextPageToken")
            if not next_page_token:
                break

        except Exception as e:
            print(f"Error fetching videos: {e}")
            time.sleep(5)  # 잠시 대기 후 다시 시도

    return videos

# JSON 파일로 저장하는 함수
def save_to_json(data, filename):
    # data 폴더가 존재하지 않으면 생성
    if not os.path.exists('data'):
        os.makedirs('data')
    
    # 파일 경로를 data 폴더 아래로 설정
    filepath = os.path.join('data', filename)
    
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

# 실행
all_videos = {}

for category_name, category_id in CATEGORIES.items():
    print(f"Fetching trending videos for category: {category_name}")
    videos = fetch_trending_videos(category_id, region_code="KR", max_results=200)
    all_videos[category_name] = videos
    print(f"비디오 {len(videos)} 개 카테고리: {category_name} fetch 완료.")

# 결과를 하나의 JSON 파일로 저장
output_file = "raw_video_data.json"
save_to_json(all_videos, output_file)
print(f"데이터 저장 : data/{output_file}'")

# 결과 출력 예시
for category, videos in all_videos.items():
    print(f"\nCategory: {category}")
    for video in videos[:5]:
        print(f" - {video['title']} ({video['video_id']}), 조회수: {video['view_count']} 회, 좋아요: {video['like_count']} 개")

Fetching trending videos for category: News & Politics
비디오 61 개 카테고리: News & Politics fetch 완료.
데이터 저장 : data/raw_video_data.json'

Category: News & Politics
 - 미친 듯 날뛰던 이재명. 전한길 등장에 꼭꼭 숨었다 [주말 몰아보기] (xJbj4MRPXOs), 조회수: 1008830 회, 좋아요: 89223 개
 - 尹 사건 맡은 중앙지법! 대형사건 터졌다![배승희 뉴스배송] (xYld5mZwuQU), 조회수: 842793 회, 좋아요: 141548 개
 - 이상민 입 열자 尹 '휘청' "10시 KBS" 발언의 비밀 [뉴스.zip/MBC뉴스] (EMKYRHDCDC8), 조회수: 1133126 회, 좋아요: 21969 개
 - [겸공뉴스특보] 2025년 2월 3일 월요일 (ZS7yJum20ic), 조회수: 550098 회, 좋아요: 32739 개
 - 일타강사 전한길 “비상계엄은 100% 계몽령” / 채널A / 뉴스 TOP10 (OQ5GJ0eMCKI), 조회수: 523307 회, 좋아요: 28908 개


# Merge

In [4]:
import os
import json
# json 파일을 로드하는 함수
def load_json(filename):
    """
    현재 디렉토리 내 data 폴더 아래의 JSON 파일을 로드하여 데이터를 반환.
    
    :param filename: 로드할 JSON 파일의 이름
    :return: JSON 데이터
    """
    data_folder = 'data'
    file_path = os.path.join(data_folder, filename)
    
    if not os.path.exists(file_path):
        print(f"파일 '{file_path}'이(가) 존재하지 않습니다.")
        return None

    with open(file_path, "r", encoding="utf-8") as f:
        return json.load(f)

# video데이터를 병합하는 함수
def merge_text_fields(video):
    """
    동영상의 title, description, tags를 하나의 긴 문자열로 병합.
    
    :param video: 동영상 데이터 딕셔너리
    :return: 병합된 문자열
    """
    title = video.get("title", "")
    description = video.get("description", "")
    tags = " ".join(video.get("tags", []))  # 태그 리스트를 공백으로 연결

    # 병합
    merged_text = " ".join([title, description, tags])
    return merged_text.strip()  # 공백 제거

# 카테고리별 동영상 데이터를 병합하는 함수
def merge_videos_with_category(data):
    """
    카테고리별 동영상 데이터를 병합하여 텍스트 필드를 생성.
    
    :param data: JSON 데이터
    :return: 병합된 텍스트 리스트
    """
    merged_texts_by_category = {}

    for category, videos in data.items():
        merged_texts = []
        for video in videos:
            # 병합된 텍스트 생성
            merged_text = merge_text_fields(video)
            merged_texts.append(merged_text)
            
        merged_texts_by_category[category] = merged_texts
            
    return merged_texts_by_category

# JSON 파일 경로
input_file_path = "raw_video_data.json"

# Load JSON data
data = load_json(input_file_path)

if data:
    # Merge text fields
    merged_texts_by_category = merge_videos_with_category(data)

    # 결과 출력
    for item in merged_texts_by_category["News & Politics"][:5]:
        print(item[:100])  # 100자까지만 출력

save_to_json(merged_texts_by_category, "merged_text_data.json")


미친 듯 날뛰던 이재명. 전한길 등장에 꼭꼭 숨었다 [주말 몰아보기] 영상 ‘좋아요’와 ‘구독’은 큰 힘이 됩니다

*‘굿모닝 대한민국’ 주말 몰아보기 영상 목록

00:00:00
尹 사건 맡은 중앙지법! 대형사건 터졌다![배승희 뉴스배송] [제보 및 비지니스 문의]
tatajebo@gmail.com

[변호사세요? 유튜버세요? - 도서 구입] 
교보문고 :
이상민 입 열자 尹 '휘청' "10시 KBS" 발언의 비밀 [뉴스.zip/MBC뉴스] 00:00 [단독] 이상민 "尹, '22시 KBS 생방송 있다'며 계엄 강행하려 해" (202
[겸공뉴스특보] 2025년 2월 3일 월요일 #김어준 #겸손은힘들다 #뉴스공장 #명랑사회 #홍사훈 #이재석

00:00:00 대기화면
00:09:45 홍사훈의 겸공뉴스특보
00:0
일타강사 전한길 “비상계엄은 100% 계몽령” / 채널A / 뉴스 TOP10 일타강사 전한길 “비상계엄은 100% 계몽령”

100만 명 모여달라"던 전한길, 부산 집회 참석
부산


# KoNLPY

In [9]:
from konlpy.tag import Okt

okt = Okt()


# 단어 추출
def extract_nouns(text):
    nouns = okt.nouns(text)
    return [n for n in nouns if len(n) > 1]  # 2음절 이상의 명사만 추출

# 카테고리별 단어 추출
def extract_nouns_by_category(merge_texts_by_category):
    nouns_by_category = {}

    for category, texts in merged_texts_by_category.items(): #texts는 배열
        nouns = []
        for text in texts:
            nouns.extend(extract_nouns(text))
        nouns_by_category[category] = nouns

    return nouns_by_category

# 실행

merged_texts_by_category = load_json("merged_text_data.json")

nouns_by_category = extract_nouns_by_category(merged_texts_by_category)

# 파일 저장
save_to_json(nouns_by_category, "nouns_by_category.json")

# 결과 출력
for category, nouns in nouns_by_category.items():
    print(f"\n카테고리: {category}")
    print(nouns[:10])  # 처음 10개 단어만 출력
    


카테고리: News & Politics
['이재명', '전한길', '등장', '꼭꼭', '주말', '보기', '영상', '구독', '굿모닝', '대한민국']


# Soynlp

In [ ]:
import soynlp

word_extractor = soynlp.extractor.WordExtractor()  #훈련 객체

word_extractor.train(comments) #훈련

word_scores = word_extractor.extract()

# Clean

In [7]:
import re
import emoji # type: ignore
import json


# 기본 텍스트 정제 함수
def preprocess_text(text):
    """
    텍스트 데이터를 전처리하고 문장을 반환합니다.

    Args:
        text (str): 입력 텍스트 데이터.

    Returns:
        str: 전처리된 텍스트.
    """
    if not isinstance(text, str):
        text = str(text)  # 문자열로 변환

    # 1. HTML 태그 제거
    text = re.sub(r"<[^>]+>", "", text)

    # 2. URL 제거
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)

    # 3. 이메일 제거
    text = re.sub(r"\S+@\S+\.\S+", "", text)

    # 4. 숫자 제거 (명확히 숫자만 제거)
    text = re.sub(r"\d+", "", text)

    # 5. 반복된 ㅋ, ㅎ, ㅠ, ㅜ 등 제거
    text = re.sub(r"[ㅋㅎㅠㅜ]+", "", text)

    # 6. 반복된 점(...) 제거
    text = re.sub(r"\.\.+", ".", text)

    # 7. 반복된 문자 축소 (e.g., "와아아" -> "와")
    text = re.sub(r"(.)\1{2,}", r"\1", text)

    # 8. 이모지 제거
    text = emoji.replace_emoji(text, replace="")

    # 9. 특수문자 및 영어 알파벳 제거
    text = re.sub(r"[^\w\s가-힣]", "", text)  # 영어 알파벳 포함 특수문자 제거
    text = re.sub(r"[a-zA-Z]", "", text)     # 영어 알파벳 제거

    # 10. 양쪽 공백 제거
    text = text.strip()

    return text


# 실행
input_file_path = "merged_text_data.json"
output_file_path = "preprocessed_text_data.json"

merged_texts_by_category = load_json(input_file_path)
# 카테고리별로 텍스트 전처리
preprocessed_texts_by_category = {}

for category, texts in merged_texts_by_category.items():
    preprocessed_texts = [preprocess_text(text) for text in texts]
    preprocessed_texts_by_category[category] = preprocessed_texts

# 결과를 JSON 파일로 저장
save_to_json(preprocessed_texts_by_category, output_file_path)
print(f"전처리된 데이터 저장: data/{output_file_path}")


전처리된 데이터 저장: data/preprocessed_text_data.json


## Extract KeyWords as Word

# Extract keyword

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
import json


def extract_ngrams(texts, n=2):
    """
    텍스트에서 N-그램 기반 구 단위 키워드 추출.

    Args:
        texts (list of str): 텍스트 리스트.
        n (int): N-그램 크기 (2: bigram, 3: trigram).

    Returns:
        dict: N-그램과 빈도수 딕셔너리.
    """
    vectorizer = CountVectorizer(ngram_range=(n, n))
    ngram_matrix = vectorizer.fit_transform(texts)
    ngram_counts = ngram_matrix.sum(axis=0).A1
    ngram_vocab = vectorizer.get_feature_names_out()

    # numpy.int64 -> int 변환
    return {ngram: int(count) for ngram, count in zip(ngram_vocab, ngram_counts)}


def extract_keyword_from_file(input_file, output_file, n=2):
    """
    문자열과 문자열 리스트가 섞인 데이터를 처리하여 결과를 저장합니다.

    Args:
        input_file (str): 입력 파일 경로.
        output_file (str): 출력 파일 경로.
        n (int): N-그램 크기.
    """
    # 파일 로드
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # 문자열 리스트로 변환 (리스트나 문자열 모두 처리)
    texts = []
    for item in data:
        if isinstance(item, list):  # 문자열 리스트인 경우 병합
            texts.append(" ".join(item))
        elif isinstance(item, str):  # 단일 문자열인 경우 그대로 추가
            texts.append(item)

    # N-그램 키워드 추출
    ngram_keywords = extract_ngrams(texts, n=n)

    # 결과 저장
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(ngram_keywords, f, ensure_ascii=False, indent=4)


if __name__ == "__main__":
    # 입력 및 출력 파일 경로
    input_file = "test_data/cleaned.json"  # 문자열과 문자열 리스트가 섞인 txt 파일
    output_file = "test_data/keyword.json"

    # 텍스트 파일 처리 (2-그램 추출)
    extract_keyword_from_file(input_file, output_file, n=2)
    print(f"처리된 데이터가 '{output_file}'에 저장되었습니다.")


처리된 데이터가 'test_data/keyword.json'에 저장되었습니다.


# Filter

In [21]:
# import json
# from collections import Counter



# def load_stopwords(filepath):
#     """
#     불용어 리스트를 로드합니다.

#     Args:
#         filepath (str): 불용어 파일 경로.

#     Returns:
#         set: 불용어 집합.
#     """
#     with open(filepath, 'r', encoding='utf-8') as f:
#         stopwords = {line.strip() for line in f}
#     return stopwords


# def split_filter_and_count_keywords(keywords, stopwords):
#     """
#     키워드를 단어로 나누고, 불용어를 제거하며 1번만 등장한 키워드를 필터링합니다.

#     Args:
#         keywords (dict): 키워드와 점수로 이루어진 딕셔너리.
#         stopwords (set): 불용어 집합.

#     Returns:
#         dict: 불용어 제거 및 1번만 등장한 키워드 제거 후 결과.
#     """
#     filtered_keywords = {}

#     for key, value in keywords.items():
#         # 1. 키워드를 공백으로 분리하여 단어 리스트 생성
#         words = key.split()

#         # 2. 단어들 중 불용어가 포함되어 있는지 확인
#         if any(word in stopwords for word in words):
#             continue  # 불용어가 포함된 경우 제거
        
    
#     # 단어 분리 및 출현 횟수 계산
#     word_counts = Counter()
#     for phrase, count in keywords.items():
#     words = phrase.split()  # 띄어쓰기 기준으로 단어 분리
#     word_counts.update({word: count for word in words})
       
#     return filtered_keywords


# def process_keywords(input_file, stopwords_file, output_file):
#     """
#     키워드 파일에서 불용어를 제거하고 1번 등장한 키워드를 제외한 결과를 저장합니다.

#     Args:
#         input_file (str): 입력 키워드 파일 경로.
#         stopwords_file (str): 불용어 파일 경로.
#         output_file (str): 결과 저장 경로.
#     """
#     # 불용어 로드
#     stopwords = load_stopwords(stopwords_file)

#     # 키워드 파일 로드
#     with open(input_file, 'r', encoding='utf-8') as f:
#         keywords = json.load(f)

#     # 불용어 제거 및 1번 등장한 키워드 필터링
#     filtered_keywords = split_filter_and_count_keywords(keywords, stopwords)

#     # 결과 저장
#     with open(output_file, 'w', encoding='utf-8') as f:
#         json.dump(filtered_keywords, f, ensure_ascii=False, indent=4)

#     print(f"불용어 제거 및 1번 등장 키워드 제외 후 결과가 '{output_file}'에 저장되었습니다.")


# # 실행
# if __name__ == "__main__":
#     # 파일 경로 설정
#     input_file = "test_data/keyword.json"  # 키워드 추출 결과 파일
#     stopwords_file = "test_data/stopwords-ko.txt"  # 불용어 리스트 파일
#     output_file = "test_data/filtered_keywords.json"  # 결과 저장 파일

#     # 키워드 필터링 수행
#     process_keywords(input_file, stopwords_file, output_file)


불용어 제거 및 1번 등장 키워드 제외 후 결과가 'test_data/filtered_keywords.json'에 저장되었습니다.
